In [ ]:
from models_wg import *
import pandas as pd
from pandas import DataFrame
import dateutil.parser
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
from preprocess import train_test_split
from preprocess import n_series
from datetime import datetime, timedelta 
from sklearn.metrics import mean_absolute_error
from collections import OrderedDict
import os
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [ ]:
# def plot(df,kunag,matnr,n,p,d,q,sl,folder): 
def plot(df,kunag,matnr,n,folder):
        test1 = train_test_split(df,kunag,matnr,n)[1]
        train = train_test_split(df,kunag,matnr,n)[0]
        plt.figure(figsize=(18,10))
        plt.plot(train.set_index("date")['quantity'], label='Train',marker = '.')
        plt.plot(test1.set_index("date")['quantity'], label='Test',marker = '.')
#         output1,rms1,mae1 = sarima(df,kunag,matnr,n,0,1,1)
#         output2,rms2,mae2 = croston_tsb(df,kunag,matnr,16,0.2,0.4)
#         y_hat_avg1 = output1
#         plt.plot(y_hat_avg1.set_index("date")['pred_column'], label='SARIMA_011' ,marker = '.')
#         y_hat_avg2 = output2
#         plt.plot(y_hat_avg2.set_index("date")['pred_column'], label='croston0.1_0.1' ,marker = '.')
        
        
        output1,rms1,mae1 = ses(df,kunag,matnr,n,0.05)
        output2,rms2,mae2 = ses(df,kunag,matnr,n,0.025)
        output3,rms3,mae3 = ses(df,kunag,matnr,n,0.001)
        y_hat_avg1 = output1
        plt.plot(y_hat_avg1.set_index("date")['pred_column'], label='ses_0.05' ,marker = '.')
        y_hat_avg2 = output2
        plt.plot(y_hat_avg2.set_index("date")['pred_column'], label='ses_0.025' ,marker = '.')
        y_hat_avg3 = output3
        plt.plot(y_hat_avg3.set_index("date")['pred_column'], label='ses_0.001' ,marker = '.')

#         output3,rms3,mae3 = prophet(df, kunag,matnr,n,sps)
#         output4,rms4,mae4 = croston_tsb(df,kunag,matnr,16,0.2,0.4)

#         y_hat_avg1 = output1
#         plt.plot(y_hat_avg1.set_index("date")['pred_column'], label='SARIMA_' + str(order),marker = '.')
#         y_hat_avg2 = output2
#         plt.plot(y_hat_avg2.set_index("date")['pred_column'], label='SES_' + str(sl),marker = '.')
#         y_hat_avg3 = output3
#         plt.plot(y_hat_avg3.set_index("date")['pred_column'], label='Prophet_' + str(sps),marker = '.',color = "blue")
#         output4,rms4,mae4 = croston_tsb(df,kunag,matnr,16,0.2,0.4)
#         y_hat_avg4 = output4
#         plt.plot(y_hat_avg4.set_index("date")['pred_column'], label='Croston TSM_' +str(0.2)+"_"+str(0.4),marker = '.')
#         output5,rms5,mae5 = croston_tsb(df,kunag,matnr,16,0.2,0.2)
#         y_hat_avg5 = output5
#         plt.plot(y_hat_avg5.set_index("date")['pred_column'], label='Croston TSM_' +str(0.2)+"_"+str(0.2),marker = '.')
#         output6,rms6,mae6 = croston_tsb(df,kunag,matnr,16,0.3,0.3)
#         y_hat_avg6 = output6
#         plt.plot(y_hat_avg6.set_index("date")['pred_column'], label='Croston TSM_' +str(0.3)+"_"+str(0.3),marker = '.')
#         output7,rms7,mae7 = croston_tsb(df,kunag,matnr,16,0.2,0.3)
#         y_hat_avg7 = output7
#         plt.plot(y_hat_avg7.set_index("date")['pred_column'], label='Croston TSM_' +str(0.2)+"_"+str(0.3),marker = '.')
#         output8,rms8,mae8 = croston_tsb(df,kunag,matnr,16,0.3,0.2)
#         y_hat_avg8 = output8
#         plt.plot(y_hat_avg8.set_index("date")['pred_column'], label='Croston TSM_' +str(0.3)+"_"+str(0.2),marker = '.')
        
        
        
        
        
        plt.legend(loc='best')
        plt.title("SES Comparision")
        index = str(kunag) + "-" + str(matnr)
        plt.savefig(folder +"/" + 'Graph_{}.png'.format(index), format="PNG")  
        return mae1,mae2,mae3
#         plt.show()

In [ ]:
import os, sys
import time
start = time.time()
bucket = pd.read_csv("bucket.csv")
df = pd.read_csv("4200_C005_2019_03_03.tsv", sep=',', header=None)
df.columns = ["kunag", "matnr", "date", "quantity","price"]
p,d,q = 0,1,1 # arima parameters
sl = 0.2 #smooting coeff for SES
sps = 0.1 # seasonality_prior_scale PROPHET
a,b = 0.1,0.1 #parameters for croston method
order = (p,q,d)
folder = 'model_graphs/'+"comp(SES_low)"
if not os.path.exists(folder):
        os.makedirs(folder)
n = 16
cnt = 0


main_df = pd.DataFrame(OrderedDict({"kunag" : ["kunag"],"matnr" : ["matnr"],
                       "ses0.05" : ["-"],
                       "ses0.025" : ["-"],
                       "ses0.001" : ["-"]}))

# main_df = pd.DataFrame(OrderedDict({"kunag" : ["kunag"],"matnr" : ["matnr"],
#                        "croston tsb0.2_0.4" : "-",
#                        "croston tsb0.2_0.2" : "-",
#                        "croston tsb0.3_0.3" : "-",
#                        "croston tsb0.2_0.3" : "-",
#                        "croston tsb0.3_0.2" : "-"}))

# main_df = pd.DataFrame(OrderedDict({"kunag" : ["kunag"],"matnr" : ["matnr"],
#                        "arima011" : ["-"],
#                        "arima111" : ["-"]}))


for i in range (1,500):
            kunag = int(bucket["kunag"].iloc[i])
            matnr = int(bucket["matnr"].iloc[i])
            index = str(kunag) +"_"+ str(matnr)
            cnt+=1
            print("count",cnt)
            print("index : ",index)
            mae1,mae2,mae3 = plot(df,kunag,matnr,n,folder)
            
#             result_df =pd.DataFrame(OrderedDict({"kunag" : [kunag],"matnr" : [matnr],
#                    "sarima" : [round(mae1,3)],
#                    "ses" : [round(mae2,3)],
#                    "croston tsb" : [round(mae4,3)]}))


#             result_df = pd.DataFrame(OrderedDict({"kunag" : ["kunag"],"matnr" : ["matnr"],
#                        "croston tsb0.2_0.4" : [round(mae4,3)],
#                        "croston tsb0.2_0.2" : [round(mae5,3)],
#                        "croston tsb0.3_0.3" : [round(mae6,3)],
#                        "croston tsb0.2_0.3" : [round(mae7,3)],
#                        "croston tsb0.3_0.2" : [round(mae8,3)]}))

            result_df = pd.DataFrame(OrderedDict({"kunag" : [kunag],"matnr" : [matnr],
                       "ses0.05" : [round(mae1,3)],
                       "ses0.025" : [round(mae2,3)],
                        "ses0.001" : [round(mae3,3)]}))

            print(result_df)
            main_df = main_df.append(result_df, ignore_index = True) 
            
            
            
end = time.time()
print("Time Taken : ",end - start)

In [ ]:
main_df
main_df = main_df[1:]
main_df.head()
print(main_df.shape)
data  = main_df
data = data.convert_objects(convert_numeric=True)
round(data.sum(axis = 0, skipna = True),3)

In [ ]:

main_df.head()

In [ ]:
export_csv = main_df.to_csv (r'results/comp(SES_0.05_0.025_0.001).csv',sep = ",", index = None, header=True)

In [ ]:
data = pd.read_csv("results/comp(SES_0.05_0.025_0.001).csv",sep = ",")

In [ ]:
minimum = data.idxmin(axis=1)

In [ ]:
a = 0
b = 0
c = 0
for i in range(0,len(minimum)):
    if(minimum[i]=="ses0.05"):
        a+=1
    elif(minimum[i]=="ses0.025"):
        b+=1
    elif(minimum[i]=="ses0.001"):
        c+=1
print(a)
print(b)
print(c)